In [1]:
# Set up logging

import logging

logging.basicConfig(
    format="[%(asctime)s %(levelname)s %(threadName)s %(name)s:%(funcName)s:%(lineno)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S%z",
)
logging.getLogger("council").setLevel(logging.INFO)

In [2]:
# Council imports

from council.runners import Budget
from council.contexts import AgentContext, ChatHistory
from council.agents import Agent
from council.chains import Chain
from council.llm.openai_llm_configuration import OpenAILLMConfiguration
from council.llm.openai_llm import OpenAILLM

from skills import SectionWriterSkill, OutlineWriterSkill
from controller import WritingAssistantController
from evaluator import BasicEvaluatorWithSource

In [3]:
# Read environment variables and initialize OpenAI LLM instance

import dotenv
dotenv.load_dotenv()
openai_llm = OpenAILLM(config=OpenAILLMConfiguration.from_env())

In [4]:
# Create Skills

outline_skill = OutlineWriterSkill(openai_llm)
writing_skill = SectionWriterSkill(openai_llm)

In [5]:
# Create Chains

outline_chain = Chain(
    name="Outline Writer",
    description="Write or revise the outline (i.e. section headers) of a research article in markdown format. Only use this chain when the ARTICLE OUTLINE is missing or incomplete.",
    runners=[outline_skill]
)

writer_chain = Chain(
    name="Article Writer",
    description="Write or revise specific section bodies of a research article in markdown format. Use this chain to write the main research article content.",
    runners=[writing_skill]
)

In [6]:
# Create Controller

controller = WritingAssistantController(
    openai_llm,
    top_k_execution_plan=3
)

In [7]:
# Initialize Agent

chat_history = ChatHistory()
run_context = AgentContext(chat_history)
agent = Agent(controller, [outline_chain, writer_chain], BasicEvaluatorWithSource())

In [8]:
# Run Agent

chat_history.add_user_message("Write a detailed research article about the history of video games.")
result = agent.execute(run_context, Budget(1200))

[2023-07-26 12:54:30-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-07-26 12:54:30-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="1"
[2023-07-26 12:54:36-0400 INFO MainThread council.agents.agent:execute:71] message="chain execution started" chain="Outline Writer" execution_unit="Outline Writer: Create an outline for the research article about the history of video games. The outline should include sections such as Introduction, Early History, Evolution of Video Games, Impact on Society, and Conclusion."
[2023-07-26 12:54:36-0400 INFO chain_Outline Writer_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="OutlineWriterSkill"
[2023-07-26 12:54:49-0400 INFO chain_Outline Writer_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="OutlineWriterSkill" skill_message="I've edited the outline and placed the response in the 'data'

In [9]:
print(result.messages[-1].message.message)

# The History of Video Games: A Detailed Exploration

## Introduction

The history of video games is a fascinating journey that spans over half a century, from the rudimentary games of the 1950s to the immersive and complex digital worlds of today. This article will delve into the evolution of video games, tracing their origins, the technological advancements that have shaped them, and the cultural impact they have had over the years. We will explore the key milestones in the development of video games, the influential figures who have driven this industry forward, and the trends that have defined each era. Whether you are a seasoned gamer or simply interested in the intersection of technology and culture, this exploration of video game history promises to be an enlightening read.

## The Early Days: 1950s - 1970s

The history of video games began in the 1950s when scientists started to design simple games and simulations as part of their computer science research. The first video game

In [10]:
chat_history.add_user_message("Can you please add more details in each section?")
result = agent.execute(run_context, Budget(600))

[2023-07-26 13:00:49-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-07-26 13:00:49-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="3"
[2023-07-26 13:01:07-0400 INFO MainThread council.agents.agent:execute:71] message="chain execution started" chain="Article Writer" execution_unit="Article Writer: In the "Early History of Video Games" section, add more information about the first video games, the birth of arcade games, and the advent of home consoles."
[2023-07-26 13:01:07-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="SectionWriterSkill"
[2023-07-26 13:02:47-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="SectionWriterSkill" skill_message="I've written or edited the article and placed it in the 'data' field."
[2023-07-26 13:02:47-0400 INFO MainThread co

In [11]:
print(result.messages[-1].message.message)

# The History of Video Games: A Detailed Exploration

## Introduction

The history of video games is a fascinating journey that spans over half a century, from the rudimentary games of the 1950s to the immersive and complex digital worlds of today. This article will delve into the evolution of video games, tracing their origins, the technological advancements that have shaped them, and the cultural impact they have had over the years. Understanding the history of video games is crucial for several reasons. Firstly, it provides insight into the technological advancements that have occurred over the years. From simple 2D graphics to complex 3D environments, from standalone games to massively multiplayer online games, the progression of video game technology mirrors the broader advancements in computing and digital technology. 

Secondly, the history of video games offers a unique perspective on cultural trends and shifts. Video games, like any form of media, reflect and shape the society